# Lab 2.6.2: Stable Diffusion XL - Master Image Generation

**Module:** 2.6 - Diffusion Models  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Beginner-Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Load and run Stable Diffusion XL on DGX Spark
- [ ] Master prompt engineering for better images
- [ ] Understand and use guidance scale effectively
- [ ] Leverage negative prompts to avoid artifacts
- [ ] Generate images in various styles and resolutions
- [ ] Benchmark generation performance

---

## 📚 Prerequisites

- Completed: Lab 2.6.1 (Diffusion Theory)
- Knowledge of: Basic diffusion model concepts
- Installed: `diffusers`, `transformers`, `accelerate`

---

## 🌍 Real-World Context

**Stable Diffusion XL (SDXL) is the workhorse of AI image generation:**

- **Midjourney** and **Leonardo.AI** are built on similar architectures
- **Adobe Firefly** uses diffusion for creative tools
- **Game studios** generate concept art and textures
- **Marketing teams** create ad visuals instantly

DGX Spark's 128GB unified memory lets you run SDXL at full precision - something that would normally require an expensive cloud GPU!

---

## 🧒 ELI5: How Does Text-to-Image Work?

> **Imagine you're playing Pictionary with a really talented artist friend.**
>
> 1. You say: "Draw a cat wearing a top hat"
> 2. Your friend starts with a blank, noisy canvas
> 3. They think: "What would a top-hat cat look like?"
> 4. They gradually clean up the noise, guided by your description
> 5. After many small improvements, a cat with a top hat appears!
>
> **The key components:**
> - **CLIP Text Encoder**: Translates your words into numbers the model understands
> - **U-Net**: The "artist" that removes noise guided by text
> - **VAE**: Compresses images to save memory (works in "latent space")
> - **Scheduler**: Controls how quickly/carefully to denoise

### The SDXL Architecture

```
"A cat wearing a top hat"     Random Noise
         │                         │
         ▼                         ▼
┌─────────────────┐         ┌─────────────┐
│  CLIP Text      │         │    VAE      │
│  Encoder        │         │  (Latent)   │
└────────┬────────┘         └──────┬──────┘
         │                         │
         │    ┌───────────────────┐│
         └───►│      U-Net        │◄┘
              │  (Denoises with   │
              │   text guidance)  │
              └─────────┬─────────┘
                        │ x30 steps
                        ▼
              ┌─────────────────┐
              │   VAE Decoder   │
              └────────┬────────┘
                       │
                       ▼
                🖼️ Final Image
```

---

## Part 1: Setting Up the Environment

In [ ]:
# Core imports
import torch
import gc
from pathlib import Path
import time
from datetime import datetime

# Diffusers - the main library for diffusion models
from diffusers import (
    StableDiffusionXLPipeline,
    DPMSolverMultistepScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
)

# Image display
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"Total Memory: {total_mem:.1f} GB")
    print(f"\nDGX Spark's 128GB unified memory = SDXL at full precision! 🚀")

In [ ]:
# Helper function to display images nicely
def show_image(image, title=None, figsize=(10, 10)):
    """Display a single image with optional title."""
    plt.figure(figsize=figsize)
    plt.imshow(image)
    if title:
        plt.title(title, fontsize=12, wrap=True)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def show_images_grid(images, titles=None, ncols=2, figsize=(12, 12)):
    """Display multiple images in a grid."""
    n = len(images)
    nrows = (n + ncols - 1) // ncols
    
    fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
    if nrows == 1:
        axes = [axes] if ncols == 1 else axes
    else:
        axes = axes.flatten()
    
    for i, (ax, img) in enumerate(zip(axes, images)):
        ax.imshow(img)
        if titles and i < len(titles):
            ax.set_title(titles[i], fontsize=10, wrap=True)
        ax.axis('off')
    
    # Hide empty subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def get_memory_usage():
    """Get current GPU memory usage."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        return f"Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB"
    return "No GPU"

print("Helper functions ready!")

---

## Part 2: Loading Stable Diffusion XL

SDXL is a ~7GB model. On DGX Spark, we can load it at full bfloat16 precision!

### DGX Spark Advantage

| Hardware | SDXL Loading | Quality |
|----------|--------------|----------|
| 8GB GPU | Must quantize to 4-bit | Lower quality |
| 16GB GPU | FP16 with memory optimization | Good |
| 24GB GPU | FP16 comfortable | Great |
| **DGX Spark (128GB)** | **BF16 + room for batches** | **Best** |

In [ ]:
print("Loading Stable Diffusion XL...")
print(f"Memory before loading: {get_memory_usage()}")

start_time = time.time()

# Load SDXL base model
# Using bfloat16 - native support on DGX Spark's Blackwell architecture!
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.bfloat16,  # Best for Blackwell
    use_safetensors=True,
    variant="fp16",  # Downloads the FP16 weights (smaller)
)

# Move to GPU
pipe = pipe.to(device)

# Enable memory efficient attention (optional on DGX Spark, but still good practice)
# pipe.enable_xformers_memory_efficient_attention()  # Uncomment if xformers installed

load_time = time.time() - start_time
print(f"\n✅ SDXL loaded in {load_time:.1f} seconds")
print(f"Memory after loading: {get_memory_usage()}")

In [ ]:
# Inspect the pipeline components
print("SDXL Pipeline Components:")
print("=" * 50)
print(f"\n1. Text Encoder 1 (CLIP ViT-L):")
print(f"   Parameters: {sum(p.numel() for p in pipe.text_encoder.parameters()):,}")

print(f"\n2. Text Encoder 2 (OpenCLIP ViT-bigG):")
print(f"   Parameters: {sum(p.numel() for p in pipe.text_encoder_2.parameters()):,}")

print(f"\n3. U-Net (Denoising Network):")
print(f"   Parameters: {sum(p.numel() for p in pipe.unet.parameters()):,}")

print(f"\n4. VAE (Image Encoder/Decoder):")
print(f"   Parameters: {sum(p.numel() for p in pipe.vae.parameters()):,}")

total_params = sum(
    sum(p.numel() for p in component.parameters())
    for component in [pipe.text_encoder, pipe.text_encoder_2, pipe.unet, pipe.vae]
)
print(f"\n📊 Total Parameters: {total_params:,} ({total_params/1e9:.2f}B)")

---

## Part 3: Your First Generation

Let's generate an image with a simple prompt!

In [ ]:
# Simple first generation
prompt = "A majestic lion resting on a rock in the African savanna, golden hour lighting, wildlife photography"

print(f"Generating image for:\n'{prompt}'\n")

# Set random seed for reproducibility
generator = torch.Generator(device=device).manual_seed(42)

# Generate!
start_time = time.time()

image = pipe(
    prompt=prompt,
    num_inference_steps=30,  # Number of denoising steps
    guidance_scale=7.5,      # How strongly to follow the prompt
    generator=generator,
    width=1024,              # SDXL native resolution
    height=1024,
).images[0]

gen_time = time.time() - start_time

print(f"⏱️ Generation time: {gen_time:.1f} seconds")
print(f"📐 Image size: {image.size}")

# Display the result
show_image(image, f"Prompt: {prompt[:80]}...")

### 🔍 What Just Happened?

1. **Text Encoding**: The prompt was converted to numerical embeddings by CLIP
2. **Random Noise**: Started with pure random noise in latent space (64×64×4)
3. **Denoising Loop**: 30 steps of guided noise removal
4. **VAE Decoding**: Latent → full resolution 1024×1024 image

On DGX Spark, you should see generation times around **5-8 seconds** at 1024×1024!

---

## Part 4: Prompt Engineering

### 🧒 ELI5: Good Prompts vs Bad Prompts

> Imagine giving directions to an artist:
>
> ❌ **Bad**: "Draw a cat"  
> The artist doesn't know what kind, what style, what setting...
>
> ✅ **Good**: "A fluffy orange tabby cat curled up on a velvet cushion, soft afternoon sunlight from a window, oil painting style, warm colors"  
> Now the artist knows exactly what you want!

### Prompt Structure Formula

```
[Subject], [Setting/Background], [Style], [Lighting/Mood], [Quality Modifiers]
```

**Examples:**
- Subject: "A wise old wizard"
- Setting: "in a tower filled with ancient books"
- Style: "fantasy art, digital painting"
- Lighting: "candlelit, mysterious atmosphere"
- Quality: "highly detailed, 8K resolution"

In [ ]:
# Compare different prompt styles
prompts = [
    # Basic prompt
    "a castle",
    
    # Detailed prompt
    "A medieval castle perched on a cliff overlooking the sea, dramatic sunset, fantasy art style, highly detailed",
    
    # Artistic style
    "A medieval castle, oil painting by Thomas Cole, Hudson River School style, romantic landscape, golden light",
    
    # Photorealistic
    "A medieval castle in Scotland, professional travel photography, golden hour, 8K resolution, National Geographic",
]

images = []
for i, prompt in enumerate(prompts):
    print(f"Generating {i+1}/{len(prompts)}: {prompt[:50]}...")
    generator = torch.Generator(device=device).manual_seed(42)  # Same seed for comparison
    
    img = pipe(
        prompt=prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
        width=1024,
        height=1024,
    ).images[0]
    images.append(img)

# Display comparison
titles = ["Basic", "Detailed", "Artistic", "Photorealistic"]
show_images_grid(images, titles, ncols=2, figsize=(14, 14))

print("\n💡 Notice how more detailed prompts give you more control over the output!")

### Prompt Weighting

You can emphasize or de-emphasize parts of your prompt using `(term:weight)` syntax.

- `(golden:1.3)` - 30% more attention to "golden"
- `(blur:0.5)` - 50% less attention to "blur"
- `((very important))` - Double emphasis (same as 1.1 × 1.1)

In [ ]:
# Demonstrate prompt weighting
# Note: Diffusers handles weighting through compel or manual attention
# For simplicity, we'll show the concept with careful prompt construction

base_prompt = "A cat and a dog playing in a garden"

weighted_prompts = [
    "A cat and a dog playing in a garden",
    "A cat and a dog playing in a beautiful magical garden with flowers",
    "A fluffy persian cat and a small dog playing in a garden",
    "A cat and a golden retriever dog playing in a Japanese zen garden",
]

images = []
for prompt in weighted_prompts:
    generator = torch.Generator(device=device).manual_seed(123)
    img = pipe(
        prompt=prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    images.append(img)

titles = ["Basic", "Emphasize Garden", "Emphasize Cat", "Emphasize Dog & Setting"]
show_images_grid(images, titles, ncols=2, figsize=(14, 14))

---

## Part 5: Guidance Scale Experiments

### 🧒 ELI5: What is Guidance Scale?

> Imagine the AI is an artist who has their own creative ideas:
>
> - **Low guidance (1-4)**: "Here's what I think you meant, with my own artistic flair"
>   - More creative, may deviate from prompt
>   - Softer, more painterly results
>
> - **Medium guidance (7-9)**: "I'll follow your description closely, but add some creativity"
>   - Best balance for most use cases
>   - Default is typically 7.5
>
> - **High guidance (12+)**: "I'll do EXACTLY what you said, nothing more"
>   - Very literal interpretation
>   - Can become oversaturated or distorted

### The Math (Classifier-Free Guidance)

```
output = unconditional_output + guidance_scale × (conditional_output - unconditional_output)
```

Higher scale = more "push" toward the text-conditioned output.

In [ ]:
# Test different guidance scales
prompt = "A cyberpunk city at night, neon lights reflecting on wet streets, futuristic architecture"

guidance_scales = [1.5, 5.0, 7.5, 12.0, 20.0]
images = []

print(f"Prompt: {prompt}\n")

for gs in guidance_scales:
    print(f"Generating with guidance_scale={gs}...")
    generator = torch.Generator(device=device).manual_seed(42)
    
    img = pipe(
        prompt=prompt,
        num_inference_steps=25,
        guidance_scale=gs,
        generator=generator,
    ).images[0]
    images.append(img)

# Display comparison
titles = [f"Scale: {gs}" for gs in guidance_scales]
show_images_grid(images, titles, ncols=3, figsize=(15, 10))

print("\n📊 Observations:")
print("  - 1.5: Very soft, dreamy, may not follow prompt well")
print("  - 5.0: Creative interpretation with good prompt following")
print("  - 7.5: Balanced (recommended default)")
print("  - 12.0: Strong prompt adherence, may oversaturate")
print("  - 20.0: Often too extreme, artifacts may appear")

---

## Part 6: Negative Prompts

### 🧒 ELI5: What are Negative Prompts?

> Telling an artist what NOT to do:
>
> "Draw me a portrait, but:
> - Don't make it blurry
> - Don't add any watermarks
> - Don't make the hands weird"
>
> This helps avoid common issues!

### Common Negative Prompt Patterns

**For photorealistic images:**
```
blurry, low quality, distorted, deformed, ugly, bad anatomy, watermark, text, signature
```

**For portraits:**
```
bad hands, extra fingers, missing fingers, bad proportions, cross-eyed, mutated
```

**For artistic styles:**
```
photo, photograph, realistic, 3D render, CGI
```

In [ ]:
# Compare with and without negative prompts
prompt = "A portrait of a wise old wizard with a long beard, fantasy art"

negative_prompts = [
    None,  # No negative prompt
    "blurry, low quality",
    "blurry, low quality, bad anatomy, deformed, ugly, mutated",
    "blurry, low quality, bad anatomy, deformed, ugly, mutated, bad hands, extra fingers, watermark, text, signature, amateur",
]

labels = [
    "No Negative",
    "Basic Negative",
    "Standard Negative",
    "Comprehensive Negative",
]

images = []
for neg in negative_prompts:
    print(f"Generating with negative: {str(neg)[:50]}...")
    generator = torch.Generator(device=device).manual_seed(88)
    
    img = pipe(
        prompt=prompt,
        negative_prompt=neg,
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    images.append(img)

show_images_grid(images, labels, ncols=2, figsize=(14, 14))

print("\n💡 Tip: Build your own 'negative prompt library' for different use cases!")

---

## Part 7: Different Art Styles

SDXL excels at generating images in various artistic styles. Let's explore!

In [ ]:
# Same subject, different styles
subject = "A serene mountain lake at sunset"

styles = {
    "Photorealistic": f"{subject}, professional landscape photography, 8K, National Geographic",
    "Oil Painting": f"{subject}, oil painting, impressionist style, thick brushstrokes, like Monet",
    "Watercolor": f"{subject}, delicate watercolor painting, soft washes, artistic",
    "Anime": f"{subject}, anime style, Studio Ghibli inspired, vibrant colors, detailed",
    "Digital Art": f"{subject}, digital art, concept art, artstation trending, fantasy",
    "Pixel Art": f"{subject}, pixel art style, 16-bit, retro game aesthetic, nostalgic",
}

images = []
for style_name, prompt in styles.items():
    print(f"Generating: {style_name}...")
    generator = torch.Generator(device=device).manual_seed(42)
    
    img = pipe(
        prompt=prompt,
        negative_prompt="blurry, low quality, distorted",
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    images.append(img)

show_images_grid(images, list(styles.keys()), ncols=3, figsize=(15, 10))

### ✋ Try It Yourself: Create Your Own Style Gallery

Choose a subject and generate it in 4 different styles!

<details>
<summary>💡 Style Ideas</summary>

- "Art Deco poster style, geometric patterns"
- "Ukiyo-e Japanese woodblock print"
- "Steampunk illustration, copper and brass tones"
- "Pop art, Andy Warhol inspired, bold colors"
- "Pencil sketch, detailed linework, sketchbook style"
- "3D render, Pixar style, cute and colorful"
- "Gothic dark fantasy, detailed, dramatic lighting"
</details>

In [ ]:
# YOUR CODE HERE: Create your own style gallery
# Choose a subject and experiment with different styles!

my_subject = ""  # Your subject here
my_styles = {
    # "Style Name": "full prompt",
}

# Generate and display...


---

## Part 8: Different Schedulers

The **scheduler** (also called sampler) controls the denoising process.
Different schedulers can give different results and speeds.

In [ ]:
# Compare different schedulers
from diffusers import (
    DDPMScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    EulerDiscreteScheduler,
    DPMSolverMultistepScheduler,
)

prompt = "A magical forest with glowing mushrooms, fantasy art, ethereal lighting"

schedulers = {
    "Euler": EulerDiscreteScheduler.from_config(pipe.scheduler.config),
    "Euler Ancestral": EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config),
    "DPM++ 2M": DPMSolverMultistepScheduler.from_config(pipe.scheduler.config),
    "DDIM": DDIMScheduler.from_config(pipe.scheduler.config),
}

images = []
times = []

for name, scheduler in schedulers.items():
    print(f"Testing {name} scheduler...")
    pipe.scheduler = scheduler
    generator = torch.Generator(device=device).manual_seed(42)
    
    start = time.time()
    img = pipe(
        prompt=prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        generator=generator,
    ).images[0]
    elapsed = time.time() - start
    
    images.append(img)
    times.append(elapsed)

# Display with timing info
titles = [f"{name}\n({t:.1f}s)" for name, t in zip(schedulers.keys(), times)]
show_images_grid(images, titles, ncols=2, figsize=(14, 14))

# Reset to default scheduler
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

print("\n📊 Scheduler Comparison:")
print("  - Euler: Fast, good quality, deterministic")
print("  - Euler Ancestral: Adds randomness, more creative")
print("  - DPM++ 2M: Very fast, high quality, recommended!")
print("  - DDIM: Classic, consistent, slightly slower")

---

## Part 9: Performance Benchmarking

Let's measure DGX Spark's performance across different configurations.

In [ ]:
# Benchmark different configurations
def benchmark(pipe, prompt, steps, width, height, num_runs=3):
    """Run benchmark and return average time."""
    times = []
    
    for _ in range(num_runs):
        generator = torch.Generator(device=device).manual_seed(42)
        torch.cuda.synchronize()
        start = time.time()
        
        _ = pipe(
            prompt=prompt,
            num_inference_steps=steps,
            guidance_scale=7.5,
            generator=generator,
            width=width,
            height=height,
        )
        
        torch.cuda.synchronize()
        times.append(time.time() - start)
    
    return sum(times) / len(times)

# Use DPM++ for fastest results
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

prompt = "A beautiful sunset over the ocean, photorealistic"

print("🚀 DGX Spark SDXL Benchmark")
print("=" * 50)

# Benchmark different configurations
configs = [
    ("512×512, 20 steps", 20, 512, 512),
    ("768×768, 25 steps", 25, 768, 768),
    ("1024×1024, 25 steps", 25, 1024, 1024),
    ("1024×1024, 30 steps", 30, 1024, 1024),
    ("1024×1024, 50 steps", 50, 1024, 1024),
]

results = []
for name, steps, w, h in configs:
    avg_time = benchmark(pipe, prompt, steps, w, h)
    results.append((name, avg_time))
    print(f"  {name}: {avg_time:.2f}s")

print("\n" + "=" * 50)
print(f"Memory Usage: {get_memory_usage()}")

---

## ⚠️ Common Mistakes

### Mistake 1: Prompts Too Short

```python
# ❌ Wrong: Too vague
prompt = "a dog"

# ✅ Right: Detailed and specific
prompt = "A golden retriever puppy playing in autumn leaves, soft afternoon light, shallow depth of field, professional pet photography"
```

### Mistake 2: Extreme Guidance Scale

```python
# ❌ Wrong: Too high, causes artifacts
guidance_scale = 30.0

# ✅ Right: Stay in reasonable range
guidance_scale = 7.5  # Good default
# Use 5-9 for most cases, up to 12 for strong prompt adherence
```

### Mistake 3: Too Few Steps

```python
# ❌ Wrong: Too few, blurry results
num_inference_steps = 5

# ✅ Right: Enough steps for quality
num_inference_steps = 25  # Good balance
# Use 20-30 for most cases, 50 for maximum quality
```

### Mistake 4: Not Using Negative Prompts

```python
# ❌ Wrong: No negative prompt
image = pipe(prompt=prompt).images[0]

# ✅ Right: Always use negative prompts
image = pipe(
    prompt=prompt,
    negative_prompt="blurry, low quality, distorted, watermark",
).images[0]
```

---

## 🎉 Checkpoint

You've learned:
- ✅ How to load and run SDXL on DGX Spark
- ✅ Prompt engineering techniques for better images
- ✅ How guidance scale affects generation
- ✅ Using negative prompts to avoid artifacts
- ✅ Generating images in various artistic styles
- ✅ Comparing different schedulers
- ✅ Benchmarking performance

---

## 🚀 Challenge (Optional)

### Challenge 1: Prompt Battle
Create the best prompt for "a cozy reading nook". Compare 5 variations and pick the winner!

### Challenge 2: Style Transfer
Generate the same scene (e.g., "a coffee shop") in 10 different famous artists' styles.

### Challenge 3: Optimal Settings
Find the optimal guidance scale and step count for photorealistic portraits vs. anime art.

---

## 📖 Further Reading

- [SDXL Paper](https://arxiv.org/abs/2307.01952) - "SDXL: Improving Latent Diffusion Models"
- [Diffusers Documentation](https://huggingface.co/docs/diffusers) - Official docs
- [Prompt Engineering Guide](https://stable-diffusion-art.com/prompt-guide/) - Comprehensive tips
- [CivitAI](https://civitai.com/) - Community models and prompts

---

## 🧹 Cleanup

In [ ]:
# Clean up GPU memory
del pipe
gc.collect()
torch.cuda.empty_cache()
print("GPU memory cleared!")
print(f"Memory after cleanup: {get_memory_usage()}")

---

## Next Steps

Now that you've mastered basic SDXL generation, proceed to:

**Lab 2.6.3: ControlNet Workshop** - Learn to guide image generation with edge maps, poses, and depth!

You'll learn:
- Using Canny edges for structural control
- Pose-controlled character generation
- Depth-based composition
- Creating consistent characters across images